ใน ep นี้เราจะ

# 0. Install

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Install Library ที่จำเป็น

In [2]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


เช็ค GPU

In [3]:
! nvidia-smi

Fri Dec 13 10:21:37 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 1. Import

Import Library ที่จะใช้ ในที่นี้คือ fastai และ fastai.text 

In [0]:
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.callbacks.mem import *

In [0]:
# import fastai.utils.collect_env
# fastai.utils.collect_env.show_install()

สำหรับ [Google Colab](https://www.bualabs.com/archives/1687/what-is-colab-open-jupyter-notebook-in-github-on-google-colab-create-open-in-colab-button-colab-ep-1/) เราจะกำหนด path ที่เก็บ DataSet และ Mount Google Drive 

In [6]:
dataset = '26i-giga-fren'

# Google Colab
config_path = Path('/content/drive')
data_path_base = Path('/content/datasets/')

data_path = data_path_base/dataset

from google.colab import drive
drive.mount(str(config_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Dataset

ในเคสนี้เราจะใช้ Human Numbers คือ รายการตัวเลข เขียนเป็นภาษาอังกฤษ 

In [7]:
data_path

PosixPath('/content/datasets/26i-giga-fren')

In [8]:
! wget https://s3.amazonaws.com/fast-ai-nlp/giga-fren.tgz -P {data_path}

--2019-12-13 09:33:10--  https://s3.amazonaws.com/fast-ai-nlp/giga-fren.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.171.165
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.171.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2598183296 (2.4G) [application/x-tar]
Saving to: ‘/content/datasets/26i-giga-fren/giga-fren.tgz.1’

giga-fren.tgz.1     100%[===================>]   2.42G  45.0MB/s    in 58s     

2019-12-13 09:34:08 (43.0 MB/s) - ‘/content/datasets/26i-giga-fren/giga-fren.tgz.1’ saved [2598183296/2598183296]



In [0]:
! tar xf {data_path}/giga-fren.tgz -C {data_path}

In [8]:
path = data_path/'giga-fren'
path.ls()

[PosixPath('/content/datasets/26i-giga-fren/giga-fren/giga-fren.release2.fixed.fr'),
 PosixPath('/content/datasets/26i-giga-fren/giga-fren/giga-fren.release2.fixed.en')]

In [9]:
! head {path}/giga-fren.release2.fixed.en

Changing Lives | Changing Society | How It Works | Technology Drives Change Home | Concepts | Teachers | Search | Overview | Credits | HHCC Web | Reference | Feedback Virtual Museum of Canada Home Page
Site map
Feedback
Credits
Français
What is light ?
The white light spectrum Codes in the light The electromagnetic spectrum Emission spectra Absorption spectra Light-years Light pollution
The sky of the first inhabitants A contemporary vison of the Universe Astronomy for everyone
Cartoon
Links


In [10]:
! head {path}/giga-fren.release2.fixed.fr

Il a transformé notre vie | Il a transformé la société | Son fonctionnement | La technologie, moteur du changement Accueil | Concepts | Enseignants | Recherche | Aperçu | Collaborateurs | Web HHCC | Ressources | Commentaires Musée virtuel du Canada
Plan du site
Rétroaction
Crédits
English
Qu’est-ce que la lumière?
La découverte du spectre de la lumière blanche Des codes dans la lumière Le spectre électromagnétique Les spectres d’émission Les spectres d’absorption Les années-lumière La pollution lumineuse
Le ciel des premiers habitants La vision contemporaine de l'Univers L’astronomie pour tous
Bande dessinée
Liens


# 3. Preprocess

In [0]:
def readfile(d): return [o.strip() for o in open(path/d).readlines()]

In [0]:
# fr = readfile(path/'giga-fren.release2.fixed.fr')
# len(fr)

In [0]:
# en = readfile(path/'giga-fren.release2.fixed.en')
# len(en)

ใช้ [RegEx](https://www.bualabs.com/archives/3070/what-is-regular-expression-regex-regexp-teach-how-to-regex-python-nlp-ep-7/) สกัด เอาเฉพาะประโยคคำถามภาษาอังกฤษออกมา ที่ขึ้นต้นด้วย "Wh" และ ลงท้ายด้วย "?" และประโยคภาษาฝรั่งเศสที่คู่กัน ที่ลงท้ายด้วย ?

In [0]:
re_eq = re.compile('^(Wh[^?.!]+\?)')
re_fq = re.compile('^([^?.!]+\?)')
en_fname = path/'giga-fren.release2.fixed.en'
fr_fname = path/'giga-fren.release2.fixed.fr'

In [0]:
lines = ((re_eq.search(eq), re_fq.search(fq))
            for eq, fq in zip(open(en_fname, encoding='utf-8'), open(fr_fname, encoding='utf-8')))
qs = [(e.group(), f.group()) for e, f in lines if e and f]

In [0]:
qs = [(q1, q2) for q1, q2 in qs]

df = pd.DataFrame({'fr': [q[1] for q in qs], 'en': [q[0] for q in qs]}, columns=['en', 'fr'])
df.to_csv(path/'questions_easy.csv')